我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」。

## 3-1 初始準備

基本上和之前是一樣的, 我們就不再說明。

    %env KERAS_BACKEND=tensorflow

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

## 3-2 讀入 IMDB 電影數據庫

今天我們要評入 IMDB 電影數據庫影評的部份。

In [3]:
from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
# save np.load
#np_load_old = np.load

# modify the default parameters of np.load
#np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
#np.load = np_load_old

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

要注意這裡我們限制只選「最常用」1 萬字, 也就是超過這範圍的就當不存在。這是文字分析常會做的事。

In [5]:
print('訓練總筆數:', len(x_train))
print('測試總筆數:', len(x_test))

訓練總筆數: 25000
測試總筆數: 25000


### 輸入資料部份

我們來看一下輸入部份長什麼樣子?

In [6]:
for n in x_train[24999]:
    print(n, end=' ')

1 17 6 194 337 7 4 204 22 45 254 8 106 14 123 4 2 270 2 5 2 2 732 2098 101 405 39 14 1034 4 1310 9 115 50 305 12 47 4 168 5 235 7 38 111 699 102 7 4 4039 9245 9 24 6 78 1099 17 2345 2 21 27 9685 6139 5 2 1603 92 1183 4 1310 7 4 204 42 97 90 35 221 109 29 127 27 118 8 97 12 157 21 6789 2 9 6 66 78 1099 4 631 1191 5 2642 272 191 1070 6 7585 8 2197 2 2 544 5 383 1271 848 1468 2 497 2 8 1597 8778 2 21 60 27 239 9 43 8368 209 405 10 10 12 764 40 4 248 20 12 16 5 174 1791 72 7 51 6 1739 22 4 204 131 9 

注意這其實是一個 list 而不是 array, 原因是每筆資料 (每段影評) 長度自然是不一樣的! 我們檢查一下前 10 筆的長度就可以知道。

In [7]:
len(x_train[24999])

153

In [8]:
len(x_train[9982])

156

In [9]:
len(x_train[9487])

104

最後要說明的是, 在每筆輸入資料的數字都代表英文的一個單字。編號方式是在我們資料庫裡所有文字的排序: 也就是出現頻率越高, 代表的數字就越小。

### 輸出資料部份

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [10]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [11]:
y_train[24999]

0

### 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [12]:
from keras.preprocessing import sequence

In [13]:
x_train = sequence.pad_sequences(x_train, maxlen=150)
x_test = sequence.pad_sequences(x_test, maxlen=150)

In [14]:
x_train.shape

(25000, 150)

至此我們可以來寫我們的第一個 RNN 了!

## 09-03 打造你的 RNN

這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

### 決定神經網路架構

* 先將 10000 維的文字壓到 N 維
* 然後用 K 個 LSTM 神經元做隱藏層
* 最後一個 output, 直接用 sigmoid 送出

### 建構我們的神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (N)。

In [18]:
N = 3 # 文字要壓到 N 維
K = 12 # LSTM 有 K 個神經元

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [20]:
model = Sequential()

In [21]:
model.add(Embedding(10000, N))

LSTM 層, 我們做 K 個 LSTM Cells。

In [22]:
model.add(LSTM(K))

單純透過 sigmoid 輸出。

In [23]:
model.add(Dense(1, activation='sigmoid'))

### 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [24]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 3)           30000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 12)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 30,781
Trainable params: 30,781
Non-trainable params: 0
_________________________________________________________________


In [23]:
(4*7 + 4)*K

128

## 09-04 訓練

我們用的 embedding 中, 會被 batch_size 影響輸入。輸入的 shape 會是

    (batch_size, 每筆上限)
    
也就是 (32,100) 輸出是 (32,100,128), 其中 128 是我們決定要壓成幾維的向量。

In [26]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=5)

/Users/yenlung/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
25000/25000 [==============================] - 42s 2ms/step - loss: 0.4453 - accuracy: 0.7733
Epoch 2/5
25000/25000 [==============================] - 43s 2ms/step - loss: 0.2541 - accuracy: 0.9020
Epoch 3/5
25000/25000 [==============================] - 40s 2ms/step - loss: 0.2001 - accuracy: 0.9266
Epoch 4/5
25000/25000 [==============================] - 40s 2ms/step - loss: 0.1672 - accuracy: 0.9396
Epoch 5/5
25000/25000 [==============================] - 39s 2ms/step - loss: 0.1377 - accuracy: 0.9521


## 09-05 檢視結果

### 分數

我們照例來看看測試資料的分數。

In [27]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 7s 283us/step


In [28]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.36388471513748166
測試資正確率 = 0.852400004863739


### 儲存結果

這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。

In [27]:
model_json = model.to_json()
open('imdb_model_arch.json',
     'w').write(model_json)
model.save_weights('imdb_model_weights.h5')